In [18]:
import pandas as pd
import numpy as np
sale_train = pd.read_csv("../Data/SaleTrain.csv")
repair_train = pd.read_csv("../Data/RepairTrain.csv")

In [19]:
sale_train

,module_category,component_category,year/month,number_sale
0,M4,P27,2005/1,0
1,M4,P27,2005/5,1042
2,M4,P27,2005/9,1677
3,M4,P27,2005/10,918
4,M4,P27,2005/11,0
...,...,...,...,...
223195,M1,P16,2006/10,0
223196,M3,P16,2006/11,1932
223197,M3,P16,2007/5,0
223198,M3,P16,2007/7,298


In [20]:
sale_group = ["module_category", "component_category", "year/month"]
sale = sale_train.groupby(sale_group).sum("number_sale").reset_index()
sale

,module_category,component_category,year/month,number_sale
0,M0,P01,2006/10,72052
1,M0,P01,2006/11,86662
2,M0,P01,2006/12,57560
3,M0,P01,2006/2,437
4,M0,P01,2006/3,52299
...,...,...,...,...
4583,M9,P31,2006/5,24117
4584,M9,P31,2006/6,11794
4585,M9,P31,2006/7,22435
4586,M9,P31,2006/8,19476


In [21]:
repair_train

,module_category,component_category,year/month(sale),year/month(repair),number_repair
0,M6,P16,2007/9,2009/4,1
1,M2,P30,2007/9,2009/8,1
2,M1,P12,2006/10,2008/2,2
3,M1,P30,2006/5,2007/7,1
4,M3,P06,2007/8,2007/12,1
...,...,...,...,...,...
510276,M2,P20,2007/4,2008/6,1
510277,M3,P15,2007/1,2008/10,1
510278,M7,P06,2006/6,2007/10,1
510279,M3,P28,2006/11,2009/1,1


In [22]:
repair_group = ["module_category", "component_category", "year/month(sale)", "year/month(repair)"]
repair = repair_train.groupby(repair_group).sum("number_repair").reset_index()
repair

,module_category,component_category,year/month(sale),year/month(repair),number_repair
0,M1,P02,2006/1,2007/6,1
1,M1,P02,2006/10,2006/12,4
2,M1,P02,2006/10,2007/1,1
3,M1,P02,2006/10,2007/10,2
4,M1,P02,2006/10,2007/11,1
...,...,...,...,...,...
54383,M9,P31,2006/9,2007/3,1
54384,M9,P31,2006/9,2008/2,1
54385,M9,P31,2006/9,2008/7,1
54386,M9,P31,2006/9,2008/8,1


In [23]:
input_table = sale.merge(repair, left_on=["module_category", "component_category", "year/month"], right_on=["module_category", "component_category", "year/month(sale)"])
input_table.drop("year/month", axis=1, inplace=True)
input_table

,module_category,component_category,number_sale,year/month(sale),year/month(repair),number_repair
0,M1,P02,52448,2006/10,2006/12,4
1,M1,P02,52448,2006/10,2007/1,1
2,M1,P02,52448,2006/10,2007/10,2
3,M1,P02,52448,2006/10,2007/11,1
4,M1,P02,52448,2006/10,2007/12,1
...,...,...,...,...,...,...
52252,M9,P31,22612,2006/9,2007/3,1
52253,M9,P31,22612,2006/9,2008/2,1
52254,M9,P31,22612,2006/9,2008/7,1
52255,M9,P31,22612,2006/9,2008/8,1


In [24]:
input_table.dtypes

module_category       object
component_category    object
number_sale            int64
year/month(sale)      object
year/month(repair)    object
number_repair          int64
dtype: object

In [25]:
# convert datetime columns to datetime format
input_table['year/month(sale)'] = pd.to_datetime(input_table['year/month(sale)'])
input_table['year/month(repair)'] = pd.to_datetime(input_table['year/month(repair)'])
input_table['months_after_sold'] = ((input_table['year/month(repair)'] - input_table['year/month(sale)'])/np.timedelta64(1, 'M')).astype(int)
input_table['sale_date'] = ((input_table['year/month(sale)'] - input_table['year/month(sale)'].min())/np.timedelta64(1, 'M')).astype(int)
input_table['repair_date'] = ((input_table['year/month(repair)'] - input_table['year/month(repair)'].min())/np.timedelta64(1, 'M')).astype(int)
input_table

,module_category,component_category,number_sale,year/month(sale),year/month(repair),number_repair,months_after_sold,sale_date,repair_date
0,M1,P02,52448,2006-10-01,2006-12-01,4,2,20,21
1,M1,P02,52448,2006-10-01,2007-01-01,1,3,20,22
2,M1,P02,52448,2006-10-01,2007-10-01,2,11,20,31
3,M1,P02,52448,2006-10-01,2007-11-01,1,13,20,32
4,M1,P02,52448,2006-10-01,2007-12-01,1,13,20,33
...,...,...,...,...,...,...,...,...,...
52252,M9,P31,22612,2006-09-01,2007-03-01,1,5,19,24
52253,M9,P31,22612,2006-09-01,2008-02-01,1,17,19,35
52254,M9,P31,22612,2006-09-01,2008-07-01,1,21,19,40
52255,M9,P31,22612,2006-09-01,2008-08-01,1,22,19,41


Event is if the component was sent back to repair, so we need to add one column to called “event’ and values are all 1. Then duration, which is called “months_after_sold” in Fig 1, is how long the repair of specific component was requested after sale. As for right-censor, it means we didn’t observe the event during experiment period, so that we cannot rule out the possibility of event beyond time frame. A common misunderstanding about right-censor is the target for each survival model is event or not, but actually right-censored or not. Therefore, it’s really important to find our right-censored data which requires a trick. For each module-component pair, we already know the amount of sales in total and how many repairs were requested during observed period, so the deduction of them can get how many components are still “alive” after observed period. Therefore, we define right-censored data to be the number of components which didn't require repairing during observed period here and the corresponding value in “event” column is 0. Finally, we define the number of events happening in one month as frequency.

In [32]:
input_table["event"] = 1
input_table

,module_category,component_category,number_sale,year/month(sale),year/month(repair),number_repair,months_after_sold,sale_date,repair_date,event
0,M1,P02,52448,2006-10-01,2006-12-01,4,2,20,21,1
1,M1,P02,52448,2006-10-01,2007-01-01,1,3,20,22,1
2,M1,P02,52448,2006-10-01,2007-10-01,2,11,20,31,1
3,M1,P02,52448,2006-10-01,2007-11-01,1,13,20,32,1
4,M1,P02,52448,2006-10-01,2007-12-01,1,13,20,33,1
...,...,...,...,...,...,...,...,...,...,...
52252,M9,P31,22612,2006-09-01,2007-03-01,1,5,19,24,1
52253,M9,P31,22612,2006-09-01,2008-02-01,1,17,19,35,1
52254,M9,P31,22612,2006-09-01,2008-07-01,1,21,19,40,1
52255,M9,P31,22612,2006-09-01,2008-08-01,1,22,19,41,1


In [36]:
final_repair_date = round((input_table['year/month(repair)'].max() - input_table['year/month(repair)'].min())/np.timedelta64(1, 'M'))

def add_censor(x):
    new_row = pd.DataFrame(x.iloc[-1, :]).transpose()
    new_row['repair_date'] = final_repair_date
    new_row['months_after_sold'] = new_row['repair_date'] - new_row['sale_date']
    new_row['event'] = 0
    new_row['number_repair'] = new_row['number_sale'] - sum(x['number_repair'])
    return pd.concat([x, new_row], axis = 0)

df_input = input_table.groupby(['module_category', 'component_category', 'sale_date']).apply(add_censor).reset_index(drop=True)
df_input = df_input.loc[:, ['module_category', 'component_category', 'sale_date', 'repair_date', 'months_after_sold', 'number_sale', 'event', 'number_repair']]
df_input.rename(columns={'number_repair': 'freq'}, inplace=True)
df_input

,module_category,component_category,sale_date,repair_date,months_after_sold,number_sale,event,freq
0,M1,P02,13,20,8,2856,1,1
1,M1,P02,13,21,9,2856,1,1
2,M1,P02,13,18,6,2856,1,2
3,M1,P02,13,29,17,2856,1,2
4,M1,P02,13,34,22,2856,1,1
...,...,...,...,...,...,...,...,...
54609,M9,P31,19,35,17,22612,1,1
54610,M9,P31,19,40,21,22612,1,1
54611,M9,P31,19,41,22,22612,1,1
54612,M9,P31,19,42,24,22612,1,1


In [ ]:
from lifelines import NelsonAalenFitter
from matplotlib import pyplot as plt
naf = NelsonAalenFitter(nelson_aalen_smoothing=False)
comp_hazard = pd.DataFrame(columns=['component_category', 'months_after_sold', 'hazard'])
for component in df_input['component_category'].unique():
    print('Current component %s' % component)
    cur_df = pd.DataFrame(columns=['component_category', 'months_after_sold', 'hazard'])
    df_input_specific = df_input[df_input['component_category'].str.match(component)]
    naf.fit(durations=df_input_specific['months_after_sold'], event_observed=df_input_specific['event'], weights=df_input_specific['freq'])
    cur_df['months_after_sold'] = naf.cumulative_hazard_.index
    cur_df['hazard'] = naf.cumulative_hazard_.diff().fillna(0).values
    cur_df['component_category'] = component
    comp_hazard = pd.concat([comp_hazard, cur_df])
    naf.plot_hazard(bandwidth=3)
plt.show()